In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys
import logging

import requests
from tqdm import tqdm_notebook
tqdm = tqdm_notebook

import vigra
import numpy as np
import pandas as pd

from dvidutils import LabelMapper
from libdvid import DVIDNodeService

from neuclease.dvid import *

import h5py
from DVIDSparkServices.io_util.brick import clipped_boxes_from_grid, Grid, box_to_slicing, box_intersection
from DVIDSparkServices.io_util.brainmaps import BrainMapsVolume

In [3]:
handler = logging.StreamHandler(sys.stdout)
root_logger = logging.getLogger()
root_logger.handlers = []
root_logger.addHandler(handler)
root_logger.setLevel(logging.INFO)
logging.getLogger('kafka').setLevel(logging.WARNING)

In [4]:
pwd

'/nrs/flyem/bergs/complete-ffn-agglo'

### Download scale 5 (scale 4 as stored in BrainMaps) to hdf5

In [32]:
    bmv = BrainMapsVolume('274750196357', 'hemibrain', 'mask_round6')

# This is the shape of the other masks Kazunori has.
shape_s5_zyx = [1344, 1280, 1088]
box_s5_zyx = np.array([[0,0,0], shape_s5_zyx])

with h5py.File('/nrs/flyem/bergs/classifications-scale-5.h5', 'w') as f:
    dset = f.create_dataset('classifications_scale_5', shape=shape_s5_zyx, dtype=np.uint8)
    
    # Note: The classifications were computed at 16nm resolution,
    # so our "scale 5" is the BrainMaps "scale 4"
    bmv_box = bmv.bounding_boxes[4]
    box_to_fetch = box_intersection(bmv_box, box_s5_zyx)
    boxes = list(clipped_boxes_from_grid(bmv_box, Grid([128,128,512])))

    for box in tqdm_notebook(boxes):
        subvol = bmv.get_subvolume(box, scale=4)
        dset[box_to_slicing(*box)] = subvol.astype(np.uint8)

Attempting refresh to obtain initial access_token
INFO [2018-08-14 17:10:23,679] Attempting refresh to obtain initial access_token
Refreshing access_token
INFO [2018-08-14 17:10:23,683] Refreshing access_token


HBox(children=(IntProgress(value=0, max=330), HTML(value='')))

### Convert to PNG

In [35]:
with h5py.File('/nrs/flyem/bergs/classifications-scale-5.h5', 'r') as f:
    vol = f['classifications_scale_5'][:]

vol = vigra.taggedView(vol, 'zyx')
for z, z_slice in enumerate(tqdm_notebook(vol)):
    d = '/nrs/flyem/bergs/kazunori/classifications-scale-5'
    vigra.impex.writeImage(z_slice, f'{d}/{z:05d}.png', dtype=np.uint8)

HBox(children=(IntProgress(value=0, max=1344), HTML(value='')))

In [52]:
(bmv.bounding_box[:,::-1] * 2 // 64 * 64) // 2

array([[    0,     0,     0],
       [17184, 19840, 20672]])

In [53]:
len(bmv.bounding_boxes)

8